<a href="https://colab.research.google.com/github/MarwanAshraf22/meeting-minutes-generator/blob/main/meeting_minutes_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q requests bitsandbytes transformers sentencepiece accelerate openai torch gradio

In [ ]:
import torch
from openai import OpenAI
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import gradio as gr

In [ ]:
AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto",
    quantization_config=quant_config
)

In [ ]:
def generate_minutes(transcription, model, tokenizer, progress=gr.Progress()):
    progress(0.6, desc="Generating meeting minutes from transcript...")

    system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
    user_prompt = f"Below is an extract transcript of a meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{transcription}"

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
    outputs = model.generate(inputs, max_new_tokens=2000)
    response = tokenizer.decode(outputs[0])

    # Clean up the response, keep only the minutes
    progress(0.9, desc="Cleaning and formatting minutes...")
    response = response.split("<|end_header_id|>")[-1].strip().replace("<|eot_id|>","")

    return response

In [ ]:
def transcribe_audio(audio_path, progress=gr.Progress()):
    progress(0.3, desc="Creating transcript from audio...")

    try:
        with open(audio_path, "rb") as audio_file:
            transcription = openai.audio.transcriptions.create(
                model=AUDIO_MODEL,
                file=audio_file,
                response_format="text"
            )
            return transcription
    except Exception as e:
        return f"Error during transcription: {str(e)}"

In [ ]:
def process_upload(audio_file, progress=gr.Progress()):
    progress(0.1, desc="Starting process...")

    if audio_file is None:
        return "Please upload an audio file."

    try:
        # Check file format
        if not str(audio_file).lower().endswith('.mp3'):
            return "Please upload an MP3 file."

        # Get transcription
        transcription = transcribe_audio(audio_file)
        if transcription.startswith("Error"):
            return transcription

        # Generate minutes
        minutes = generate_minutes(transcription, model, tokenizer)
        progress(1.0, desc="Process complete!")
        return minutes

    except Exception as e:
        return f"Error processing file: {str(e)}"

In [ ]:
# Create Gradio interface

interface = gr.Interface(
    fn=process_upload,
    inputs=gr.Audio(type="filepath", label="Upload MP3 File", format="mp3"),
    outputs=gr.Markdown(label="Meeting Minutes", min_height=60),
    title="Meeting Minutes Generator",
    description="Upload an MP3 recording of your meeting to get AI-generated meeting minutes. This process may take a few minutes.",
    flagging_mode="never"
)

In [ ]:
interface.launch()